Automatic function for the correction of the jumps (more than one version): 
- Saturation function 
- Application of not of the savgol filter that adjusts a polynomial  
- Haar filter 
- Threshold 
- Clustering, beginning and final of the jump. If the distance between jumps is very small I don't save this data. 
- Offsets, with differences or fitting a lineal polynomial
- Correction
- Correccion of the peaks that are left over, only data that is removed. 

In [ ]:
top_dir = '/home/bcostanza/qubic/jumps'


dataset0 = top_dir+'/2022-07-14/2022-07-14_23.54.19__MoonScan_Speed_VE14'

import numpy as np
from qubicpack.qubicfp import qubicfp
import sys,os
import numpy as np

from qubic import fibtools as ft

import matplotlib.pyplot as plt
from pysimulators import FitsArray

from qubic import fibtools as ft
from qubic.plotters import *
from qubicpack.qubicfp import qubicfp
import qubic.demodulation_lib as dl
import qubic.sb_fitting as sbfit
from qubic.io import write_map

import matplotlib.mlab as mlab
import scipy.ndimage.filters as f

from scipy.signal import argrelextrema, find_peaks, find_peaks_cwt, savgol_filter 

In [ ]:
import bottleneck as bn
from sklearn.cluster import DBSCAN

In [ ]:
a = qubicfp()
a.read_qubicstudio_dataset(dataset0)


In [ ]:
tod = a.tod()
timeaxis = tod[0]
todarray = tod[1]


In [ ]:
init = timeaxis[0]
tt = timeaxis - init

Discard saturated TES more than 10%

In [ ]:
def saturation(todarray): 
    
    
    #returns  ok = array of True and False 
    #         bad_idx = idx of the saturated TES in the focalplane 
    #         frac_sat_pertes = fraction of the TOD saturated in the TES
    #         number = number of TES saturated more than 10% in the focalplane
    
    ok = np.ones(256,dtype=bool)
    maxis = np.max(abs(todarray), axis=1)
    upper_satval = np.max(maxis)
    lower_satval = -np.max(maxis)
    badtes = (maxis == np.max(maxis))
    number = badtes.sum() #number of TES saturated in every measurement
    bad_idx = np.array(np.where(badtes==True)) #index of the TES saturated
    bad_idx = np.reshape(bad_idx, (bad_idx.shape[1]))

    frac_sat_pertes = np.zeros((256))
    size = todarray.shape[1]

    for i in range(len(todarray)): 
        mask1 = todarray[i] == upper_satval
        mask2 = todarray[i] == lower_satval
        frac = (np.sum(mask1)+np.sum(mask2))/size
        frac_sat_pertes[i] = frac
    
        if frac_sat_pertes[i] ==0:
            ok[i] = True #good, no saturated
        elif frac_sat_pertes[i] >0.1:
            ok[i] = False #bad, saturated more than 10%
        else: 
            ok[i] = True #good, saturated but less than 10%
            
    return ok, bad_idx, frac_sat_pertes, number

In [ ]:
ok, bad_idx, frac, number = saturation(todarray)

In [ ]:
number

In [ ]:
bad_tod = np.array(np.where(ok==False))
bad_tod = np.reshape(bad_tod, (bad_tod.shape[1]))
good_tod = np.array(np.where(ok==True))
good_tod = np.reshape(good_tod, (good_tod.shape[1]))


In [ ]:
bad_tod #discarded, only index

In [ ]:
good_tod #TES that I will work, some of them have jumps 

Function that detects the jumps V1

In [ ]:
def jumps_new(todarray, size=100, treshold=0.2e6, doplot=False): 
    
    #return xc = beginning of the jump 
    #       xcf = final of the jump 
    #       dif = diference between the final of a jump and the beginning of the next jump 
    #       nc = number of clusters/number of jumps 
    
    tod_sav = savgol_filter(todarray,window_length=401,polyorder=3, mode='nearest')
    
    def haar(x, size=100):
        out = np.zeros(x.size)
        xf = bn.move_median(x, size)[size:]   
        out[size+size//2:-size+size//2] = xf[:-size] - xf[size:]
        return out
    
    #treshold using that the difference is going to be value between 0.25e6, 0.5e6, 1e6, 1.5e6, 2e6, 2.5e6
    jumps = np.abs(tod_haar) > treshold
    
    #if you don't find any jump then don't make the savgol filter and try with the unfiltered TOD
    true = np.array(np.where(jumps==False))
    if true.shape[1] == len(todarray):
        tod_haar = haar(todarray)
        jumps = np.abs(tod_haar) > treshold

    
    idx = np.arange(len(todarray))
    idx_jumps = idx[jumps] - 50 
    time_jumps = tt[jumps]
        
    
    clust = DBSCAN(eps=size//5, min_samples=1).fit(np.reshape(idx_jumps, (len(idx_jumps),1)))
    nc = np.max(clust.labels_)+1
    xc = np.zeros(nc, dtype=int) 
    xcf = np.zeros(nc, dtype=int)
    for i in range(nc):
        xc[i] = np.min(idx_jumps[clust.labels_ == i])
        xcf[i]= np.max(idx_jumps[clust.labels_ == i])
        
        
    dif = np.zeros(nc-1)
    for j in range(nc):
        if j < nc-1:
            dif[j] = xc[j+1]-xcf[j]
        
        
    if doplot == True:
        fig, ax = plt.subplots(3, figsize = (10,10))
        if true.shape[1] == len(todarray):
            ax[0].plot(tt, todarray)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        else:
            ax[0].plot(tt, tod_sav)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        
        if true.shape[1] == len(todarray):
            ax[1].plot(tt, todarray)
            ax[1].plot(tt[idx_jumps], todarray[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        else:       
            ax[1].plot(tt, tod_sav)
            ax[1].plot(tt[idx_jumps], tod_sav[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        
        ax[2].plot(tt, tod_haar, label='haar filter')
        ax[2].plot(tt[idx_jumps], tod_haar[idx_jumps], 'r.')
        ax[2].plot(tt[xc], tod_haar[xc], 'g.')
        ax[2].plot(tt[xcf], tod_haar[xcf], 'g.')
        ax[2].legend()
        
    return xc, xcf, dif, nc

In [ ]:
xc_5, xcf_5, dif_5, number_5= jumps_new(todarray[5],doplot=False)

In [ ]:
xc_31, xcf_31, dif_31, number_31 = jumps_new(todarray[31],doplot=True)

In [ ]:
xc_51, xcf_51, dif_51, number_51 = jumps_new(todarray[50],doplot=True)

In [ ]:
xc_28, xcf_28, dif_28, number_28 = jumps_new(todarray[28],doplot=True)

In [ ]:
tod_31 = todarray[31]
idx = np.arange(len(tod_31))

In [ ]:
plt.plot(idx, tod_31)
plt.plot(idx[xc_31], tod_31[xc_31], 'r.', label='cluster initial')
plt.plot(idx[xcf_31], tod_31[xcf_31], 'g.', label='cluster final')
plt.xlabel('time')
plt.ylabel('tod')
plt.legend()

In [ ]:
plt.plot(idx, tod_31)
plt.plot(idx[xc_31], tod_31[xc_31], 'r.', label='cluster initial')
plt.plot(idx[xcf_31], tod_31[xcf_31], 'g.', label='cluster final')
plt.xlabel('time')
plt.ylabel('tod')
plt.legend()

Function that redefines the number of clusters. If the distances between clusters if very small I'm going to consider as a single one.

In [ ]:
def redefine(xc, xcf, dif, number): 
    
    shape = np.shape(dif)
    if shape != 0:  
        for i in range(number-1): 
            value = dif[i]
            if value < 500.: 
                number_new = number -1 
                xc_new = np.zeros(number-1, dtype=int)
                xcf_new = np.zeros(number-1, dtype=int)
                for j in range(number-1): 
                    xc_new[j]= xc[i]
                    xcf_new[j] = xcf[i+1]
    return xc_new, xcf_new, number_new

In [ ]:
xc_new, xcf_new, number_new = redefine(xc_31,xcf_31,dif_31,number_31)

In [ ]:
number_new 

In [ ]:
xc_new

In [ ]:
plt.plot(idx, tod_31)
plt.plot(idx[xc_new], tod_31[xc_new], 'r.')
plt.plot(idx[xcf_new], tod_31[xcf_new], 'r.')

In [ ]:
#Correction of the jump calculating the offset and shifting
def corr_off(todarray, xc, xcf, number, region=100):  
    tod_new = todarray.copy()
    offset = np.zeros(number)
    for i in range(len(xc)): 
        offset[i] = np.median(todarray[xcf[i]:xcf[i]+region])-np.median(todarray[xc[i]-region:xc[i]])
    if number == 1: 
        initial = xcf[0]
        final = idx[-1]+1
        tod_new[initial:final] = todarray[initial:final] - offset[0]
    else:
        for i in range(len(xcf)-1,-1,-1):
            initial = xcf[i]
            final = idx[-1]+1
            tod_new[initial:final]=tod_new[initial:final]-offset[i]
    return tod_new, offset

In [ ]:
#Correction of the jump fitting a linear polynomial and shifting
def pol_corr(todarray, xc, xcf, number, region=100): 
    tod_new = todarray.copy()
    pol = np.zeros(len(xc))
    off = np.zeros(len(xc))
    for i in range(len(xc)):        
        tp = tt[xc[i]-region:xcf[i]+region]
        adup = todarray[xc[i]-region:xcf[i]+region]
        z = np.polyfit(tp, adup, 1)
        p = np.poly1d(z)
        pol = p(tp)
        off[i] = pol[-1]-pol[0]
    if number == 1: 
        tod_new[xcf[0]+region:] = todarray[xcf[0]+region:] - off[0]
    else: 
        for i in range(len(xcf)-1,-1,-1):
            initial = xcf[i]
            final = idx[-1]+1
            tod_new[initial:final]=tod_new[initial:final]-off[i]
    return tod_new, off

In [ ]:
tod_new_51, offset_51 = corr_off(todarray[50], xc_51, xcf_51, number_51, region=100)

In [ ]:
plt.plot(tt, tod_new_51)
plt.ylabel('tod corrected')
plt.xlabel('time')
plt.title('TES 51')

In [ ]:
tod_pol_51, offset_51 = pol_corr(todarray[50],xc_51, xcf_51, number_51, region=100)

In [ ]:
plt.plot(tt, tod_pol_51)
plt.ylabel('tod corrected')
plt.xlabel('time')
plt.title('TES 51')

In [ ]:
tod_new_28, offset_28 = corr_off(todarray[28], xc_28, xcf_28, number_28, region=50)

In [ ]:
offset_28

In [ ]:
plt.plot(tt, tod_new_28)
plt.ylabel('tod corrected')
plt.xlabel('time')
plt.title('TES 28')

In [ ]:
#replace the peak with random noise realization
def changesignal(y, xini, xend):
    # interpolation + random noise
    #yini = np.mean(y[:xini])
    #yend = np.mean(y[xend:])# - offset
    y_cor = y.copy()
    # Take the std of the signal up to idx_jump
    for i in range(len(xini)):        
        std = np.std(y[xini[i]-10:xini[i]])# * (len(y[:xini]))/(len(y[:xini])-1)
        mean = np.mean(y[xini[i]-10:xini[i]])
        ynew=np.random.normal(mean, std, len(y[xini[i]:xend[i]]))
        y_cor[xini[i]:xend[i]] = ynew
    
    return y_cor

In [ ]:
tod = todarray[good_tod]

Function that detects the jumps V2

In [ ]:
def jumps_new_2(todarray, size=100, treshold=0.2e6, doplot=False): 
    
    #return xc = beginning of the jump 
    #       xcf = final of the jump 
    #       dif = diference between the final of a jump and the beginning of the next jump 
    #       nc = number of clusters/number of jumps
    
    
    tod_sav = savgol_filter(todarray,window_length=401,polyorder=3, mode='nearest')
    
    def haar(x, size=100):
        out = np.zeros(x.size)
        xf = bn.move_median(x, size)[size:]   
        out[size+size//2:-size+size//2] = xf[:-size] - xf[size:]
        return out
    
    #treshold using that the difference is going to be a value between 0.25e6, 0.5e6, 1e6, 1.5e6, 2e6, 2.5e6
    
    tod_haar_sav = haar(tod_sav, size)
    tod_haar = haar(todarray,size)
    
    #if you don't find any jump then don't make the savgol filter and try with the unfiltered TOD
    if max(abs(tod_haar_sav)) < treshold:
        #if the tood haar also is less than 0.2, probably is not a jump 
        if max(abs(tod_haar)) < 0.2e6:
            number = 0
            print('no jump')
            return number
        else:
            jumps = np.abs(tod_haar) > treshold
            no_sav = True #don't use the savgol filter
    else:
        no_sav = False
        print('use savgol')
        jumps = np.abs(tod_haar_sav) > treshold
    
    idx = np.arange(len(todarray))
    #por el threshold que pones me parece que estas perdiendo valores por eso al indice le resto 50
    idx_jumps = idx[jumps] - 50
    time_jumps = tt[jumps]
        
    
    clust = DBSCAN(eps=size//5, min_samples=1).fit(np.reshape(idx_jumps, (len(idx_jumps),1)))
    nc = np.max(clust.labels_)+1
    xc = np.zeros(nc, dtype=int) 
    xcf = np.zeros(nc, dtype=int)
    for i in range(nc):
        xc[i] = np.min(idx_jumps[clust.labels_ == i])
        xcf[i]= np.max(idx_jumps[clust.labels_ == i])
        
    delta = xcf - xc
    print(delta)
    for i in range(len(delta)):
        if delta[i] < 80: 
            number = 0 
            print('no jump')
            return number 
        
    dif = np.zeros(nc-1)
    for j in range(nc):
        #print(j)
        if j < nc-1:
            dif[j] = xc[j+1]-xcf[j]
        
        
    if doplot == True:
        fig, ax = plt.subplots(3, figsize = (10,10))
        if no_sav:
            ax[0].plot(tt, todarray)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        else:
            ax[0].plot(tt, tod_sav)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        
        if no_sav:
            ax[1].plot(tt, todarray)
            ax[1].plot(tt[idx_jumps], todarray[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        else:       
            ax[1].plot(tt, tod_sav)
            ax[1].plot(tt[idx_jumps], tod_sav[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        
        if no_sav:            
            ax[2].plot(tt, tod_haar, label='haar filter')
            ax[2].plot(tt[idx_jumps], tod_haar[idx_jumps], 'r.')
            ax[2].plot(tt[xc], tod_haar[xc], 'g.')
            ax[2].plot(tt[xcf], tod_haar[xcf], 'g.')
            ax[2].legend()
        else:
            ax[2].plot(tt, tod_haar_sav, label='haar filter')
            ax[2].plot(tt[idx_jumps], tod_haar_sav[idx_jumps], 'r.')
            ax[2].plot(tt[xc], tod_haar_sav[xc], 'g.')
            ax[2].plot(tt[xcf], tod_haar_sav[xcf], 'g.')
            ax[2].legend()
        
    return xc, xcf, dif, nc

Loop in the entire focalplane

In [ ]:
for i in range(len(tod)):
    good = tod[i]
    locals()['result_{}'.format(i)]=jumps_new_2(good,doplot=False)

In [ ]:
result3 = jumps_new_2(tod[3], doplot=True)

Example of no jump detected

In [ ]:
plt.plot(tt, todarray[48])
plt.title('TES 49')

In [ ]:
plt.plot(tt, tod[28])

In [ ]:
result15 = jumps_new_2(tod[15], doplot=True)

In [ ]:
result5 = jumps_new_2(todarray[5], doplot=True)